In [25]:
import geopandas as gpd
import numpy as np
import statistics as st

In [2]:
canopy_e = gpd.read_file('..\datasets\segmentacao_todos\canopy_detection_result_dem.geojson')
canopy_e['id'] = canopy_e.index
canopy_r = gpd.read_file('D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/Analise_final/train_canopy_all_recortado.geojson')
canopy_r['id'] = canopy_r.index

In [13]:
canopy_r

,DN,id,layer,path,geometry
0,NaN,0,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((289046.269 7515262.288, 289046.109 7..."
1,NaN,1,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((289051.301 7515270.589, 289051.325 7..."
2,NaN,2,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((289039.897 7515256.119, 289039.888 7..."
3,NaN,3,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((289036.411 7515249.484, 289036.454 7..."
4,NaN,4,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((289036.799 7515256.399, 289036.795 7..."
...,...,...,...,...,...
2211,0.0,2211,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((280288.984 7559792.997, 280288.948 7..."
2212,0.0,2212,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((280288.984 7559796.118, 280288.984 7..."
2213,0.0,2213,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((280288.984 7559798.334, 280288.984 7..."
2214,0.0,2214,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,"POLYGON ((280288.984 7559800.528, 280288.976 7..."


In [3]:
def get_intersection(gpd_canopy_r, gpd_canopy_e):
    
    over = gpd.overlay(gpd_canopy_r, gpd_canopy_e, how='intersection')
    over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)
    over['geo_id1'] = gpd_canopy_r.loc[list(over['id_1']), 'geometry'].values
    over['geo_id2'] = gpd_canopy_e.loc[list(over['id_2']), 'geometry'].values
    
    return over
    
def calculate_matches(gpd_intersection):
    matches = gpd_intersection[over['geometry'].area >= 0.5 * gpd_intersection['geo_id1'].area]
    matches = matches.loc[matches['geometry'].contains(matches['geo_id1'].centroid)]
    
    return matches

def get_union(matches):
    
    list_union = []
    for i in matches.index:
        union = matches.loc[i, 'geo_id1'].union(matches.loc[i, 'geo_id2'])
        list_union.append(union)

    matches['union_geo'] = list_union
    matches['IoU'] = matches['geometry'].area / gpd.GeoSeries(matches['union_geo']).area
    
    return matches  

In [ ]:
## Artigo: Extraction of information about individual trees from high-spatial-resolution uav-acquired images of an orchard
## Métrica: Classes de resultado

over = get_intersection(canopy_r, canopy_e)
match = calculate_matches(over)

canopy_e['result'] = ''
canopy_e.at[match['id_2'].values, 'result'] = 'match'

# canopy_e.to_file('../datasets/segmentacao_todos/validacao_over-match.geojson', driver='GeoJSON')

In [4]:
## Artigo: Mask R-CNN refitting strategy for plant counting and sizing in uav imagery
## Metrica: mean Average Precision (mAP)

over = get_intersection(canopy_r, canopy_e)
match = calculate_matches(over)
df_final = get_union(match)
df_final

,DN,id_1,layer,path,score,detection_score,id_2,geometry,geo_id1,geo_id2,union_geo,IoU
0,NaN,0,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.996930,1752,"POLYGON ((289046.245 7515262.340, 289046.109 7...","POLYGON ((289046.269 7515262.288, 289046.109 7...","POLYGON ((289046.037 7515263.018, 289046.104 7...","POLYGON ((289046.2689655506 7515262.288380339,...",0.712919
1,NaN,1,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999827,1741,"POLYGON ((289051.371 7515268.614, 289051.370 7...","POLYGON ((289051.301 7515270.589, 289051.325 7...","POLYGON ((289051.225 7515270.334, 289051.714 7...","POLYGON ((289051.3007838524 7515270.588853614,...",0.797287
2,NaN,2,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999486,1711,"POLYGON ((289040.139 7515257.223, 289040.242 7...","POLYGON ((289039.897 7515256.119, 289039.888 7...","POLYGON ((289040.782 7515258.046, 289040.850 7...","POLYGON ((289039.8969961464 7515256.119277151,...",0.851808
3,NaN,3,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999932,1713,"POLYGON ((289036.444 7515249.554, 289036.454 7...","POLYGON ((289036.411 7515249.484, 289036.454 7...","POLYGON ((289036.373 7515249.232, 289036.440 7...","POLYGON ((289036.4106411996 7515249.48419077, ...",0.812188
4,NaN,4,train_canopy_final,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999737,1710,"POLYGON ((289039.492 7515256.260, 289039.418 7...","POLYGON ((289036.799 7515256.399, 289036.795 7...","POLYGON ((289037.297 7515256.977, 289037.366 7...","POLYGON ((289036.798595182 7515256.399043243, ...",0.748643
...,...,...,...,...,...,...,...,...,...,...,...,...
3411,0.0,2211,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999319,931,"POLYGON ((280287.926 7559794.341, 280287.935 7...","POLYGON ((280288.984 7559792.997, 280288.948 7...","POLYGON ((280287.893 7559794.208, 280287.937 7...","POLYGON ((280288.98377 7559792.996830669, 2802...",0.709030
3413,0.0,2213,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999524,922,"POLYGON ((280288.808 7559798.099, 280288.532 7...","POLYGON ((280288.984 7559798.334, 280288.984 7...","POLYGON ((280287.521 7559798.235, 280287.537 7...","POLYGON ((280288.98377 7559798.33443, 280288.9...",0.723158
3414,0.0,2212,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.998832,925,"POLYGON ((280288.788 7559795.568, 280288.772 7...","POLYGON ((280288.984 7559796.118, 280288.984 7...","POLYGON ((280288.004 7559797.000, 280288.005 7...","POLYGON ((280288.98377 7559796.11843, 280288.9...",0.628232
3416,0.0,2214,train_canopy_aguai_suav,D:/FelipeSa/OneDrive/Felipe_INPE/Segmentacao/A...,None,0.999480,923,"POLYGON ((280288.793 7559800.435, 280288.783 7...","POLYGON ((280288.984 7559800.528, 280288.976 7...","POLYGON ((280287.516 7559801.542, 280287.850 7...","POLYGON ((280288.98377 7559800.528274069, 2802...",0.710336


In [19]:
canopy_r.loc[[x for x in canopy_r.index if x not in list(df_final['id_1'])]].to_file('../datasets/segmentacao_todos/validacao_not_matches.geojson', driver='GeoJSON')

In [ ]:
df_final['geometry'].area / gpd.GeoSeries(df_final['union_geo']).area

In [23]:
all_percent = []
for i in np.arange(0.5, 0.96, 0.05):
    iou = df_final['geometry'].area / gpd.GeoSeries(df_final['union_geo']).area
    
    all_percent.append(len(iou.loc[iou>i])/2216)
    
all_percent

[0.9237364620938628,
 0.9151624548736462,
 0.9038808664259927,
 0.8826714801444043,
 0.8501805054151624,
 0.753158844765343,
 0.5487364620938628,
 0.18456678700361012,
 0.016245487364620937,
 0.0]

In [26]:
st.mean(all_percent)

0.5978339350180506

In [27]:
np.arange(0.5, 0.96, 0.05)

array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [ ]:
[len(iou.loc[iou>i]) for i in np.arange(0.5, 0.96, 0.05)]

In [ ]:
u = match.drop(['geo_id1','geo_id2'], axis=1)
u['geometry'] = u['union_geo']
u = u.drop('union_geo', axis=1)
u.to_file('../datasets/segmentacao_todos/validacao_union.geojson', driver='GeoJSON')

In [ ]:
match.iloc[167,2]

In [ ]:
match.iloc[167,3]

In [ ]:
match.iloc[167,4]

In [ ]:
intersec['geo_id1'] = Polygon()
intersec['geo_id2'] = Polygon()

for i in intersec.index:
    intersec.at[i, 'geo_id1'] = 
    

In [ ]:
cascaded_union([match.iloc[167,3], match.iloc[167,4]])

In [ ]:
i

In [ ]:
match

In [ ]:
match.loc[i, 'geo_id2']

In [ ]:
union.to_file('../datasets/segmentacao_todos/validacao_union.geojson', driver='GeoJSON')

In [ ]:
match.loc[i, 'geo_id1'].union(match.loc[i, 'geo_id2'])

In [ ]:
u

In [ ]:
list_union[2]